In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from helper import evaluate_model

In [7]:
df = pd.read_csv('data/tracks_spectral_reduced.csv')

### Data Preparation

Since we will apply regression models in this part, we have to handle categorical columns. We will either encode them or drop them if they have high cardinality.

In [8]:
# Identify categorical columns excluding 'genres' and 'genres_all'
# These columns should be dropped or encoded if necessary
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
cols_to_handle = [col for col in categorical_cols if col not in ['genres', 'genres_all']]
print(cols_to_handle)
# ['album_title', 'artist_name', 'title']
# Since these columns have high cardinality and are not useful for regression, we will drop them
cols_to_drop = cols_to_handle
df = df.drop(columns=cols_to_drop, errors='ignore')

['album_title', 'artist_name', 'title']


Now, we will analyse the target feature : duration.

In [9]:
# Outlier clipping and log transformation for 'duration'
print(df['duration'].describe().round(2).T)

# We see an extreme outlier at 11030 seconds, so we will clip durations at 2000 seconds.

# Clipping
MAX_DURATION = 2000.0
df['duration_clipped'] = np.clip(df['duration'], a_min=None, a_max=MAX_DURATION)

# log transformation (to reduce skewness for regression)
df['y_duration_log'] = np.log1p(df['duration_clipped'])

count    97288.00
mean       274.82
std        283.36
min          0.00
25%        151.00
50%        218.00
75%        306.00
max      11030.00
Name: duration, dtype: float64


Now, we define the input set and the target 'y_duration_log'.  
For the input, it will exclude the target feature (and its origins, 'duration' and 'duration_clipped'). It will exclude genres and genres_all too because they are a list of ids. Finally, it will exclude the columns 'Unnamed: 0' and the column 'track_id'.

In [10]:
# Split into inputs X and target Y
Y = df['y_duration_log']
cols_to_exclude = [
    'duration',
    'duration_clipped',
    'y_duration_log',
    'genres', 'genres_all',
    'Unnamed: 0',
    'track_id'
]
X = df.drop(columns=[col for col in cols_to_exclude if col in df.columns], errors='ignore')
print(f"Shape of X (Input Features): {X.shape}")
print(f"Shape of Y (Target): {Y.shape}")

Shape of X (Input Features): (97288, 19)
Shape of Y (Target): (97288,)


Here, we split the data into train and test data and we standarize based on X_train.

In [11]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=0.2, 
    random_state=42 
)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

Shape of X_train: (77830, 19)
Shape of X_test: (19458, 19)


In [12]:
# Standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

## Baseline : KNN Regressor

In [13]:
# Baseline : KNN Regressor

model_knn = KNeighborsRegressor(n_neighbors=5) 
model_knn.fit(X_train_scaled, Y_train)

#Predictions
Y_pred_log = model_knn.predict(X_test_scaled)

#Evaluation
rmse_log = np.sqrt(mean_squared_error(Y_test, Y_pred_log))
mae_log = mean_absolute_error(Y_test, Y_pred_log)

# We convert MAE back to seconds for interpretability
mae_seconds = np.expm1(mae_log)

# R2 Score
r2_score = model_knn.score(X_test_scaled, Y_test)

print("\nKNN Regressor Performance:")
print(f"R² Score: {r2_score:.4f}")
print(f"RMSE (Log-Seconds): {rmse_log:.4f}")
print(f"MAE (Log-Seconds): {mae_log:.4f}")
print(f"MAE (Seconds): {mae_seconds:.2f} seconds")


KNN Regressor Performance:
R² Score: 0.1408
RMSE (Log-Seconds): 0.6745
MAE (Log-Seconds): 0.4895
MAE (Seconds): 0.63 seconds


## Random Forest Regressor

In [14]:
#Random Forest Regressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf.fit(X_train_scaled, Y_train)

#Predictions
Y_pred_log = model_rf.predict(X_test_scaled)

#Evaluation
rmse_log = np.sqrt(mean_squared_error(Y_test, Y_pred_log))
mae_log = mean_absolute_error(Y_test, Y_pred_log)
mae_seconds = np.expm1(mae_log)
r2_score = model_rf.score(X_test_scaled, Y_test)

print("\nRandom Forest Regressor Performance:")
print(f"R² Score: {r2_score:.4f}")
print(f"RMSE (Log-Seconds): {rmse_log:.4f}")
print(f"MAE (Log-Seconds): {mae_log:.4f}")
print(f"MAE (Seconds): {mae_seconds:.2f} seconds")

KeyboardInterrupt: 

## Merge with Echonest

In this part, we will add features from `echonest_features.tsv`. This merge will reduce the size of the dataset to 90%.  
We will compare the performance of a random forest regressor on a small data set with more features and on a large data set but with less features.

In [15]:
# Merging with Echonest features
echonest = pd.read_csv('data/echonest_features.tsv', sep='\t')

df_complet = df.merge(echonest, on='track_id')

print(f"Old shape : {df.shape}")
print(f"New shape (after merge) : {df_complet.shape}")

Old shape : (97288, 26)
New shape (after merge) : (10552, 34)


In [16]:
categorical_cols_after = df_complet.select_dtypes(include=['object']).columns.tolist()
cols_to_handle_after = [col for col in categorical_cols_after if col not in ['genres', 'genres_all']]
print(cols_to_handle_after)
# No new categorical columns introduced

[]


In [17]:
#Verifying missing values after merging
df_complet.isnull().sum()/len(df_complet) *100

Unnamed: 0                   0.000000
track_id                     0.000000
album_tracks                 0.000000
artist_latitude              0.000000
artist_longitude             0.000000
duration                     0.000000
genre_top                    0.000000
genres                       0.000000
genres_all                   0.000000
spectral_bandwidth_max_01    0.000000
spectral_bandwidth_min_01    0.000000
spectral_bandwidth_std_01    0.000000
spectral_centroid_max_01     0.000000
spectral_centroid_min_01     0.000000
spectral_centroid_std_01     0.000000
spectral_rolloff_max_01      0.000000
spectral_rolloff_min_01      0.000000
spectral_rolloff_std_01      0.000000
artist_location_unknown      0.000000
g4_pc1                       0.000000
g5_pc1                       0.000000
g6_pc1                       0.000000
g6_pc2                       0.000000
g6_pc3                       0.000000
duration_clipped             0.000000
y_duration_log               0.000000
acousticness

In [18]:
# Impute missing values with median
cols_to_impute = ['speechiness', 'valence', 'danceability']

for col in cols_to_impute:
    median_val = df_complet[col].median()
    df_complet[col] = df_complet[col].fillna(median_val)

In [19]:
# Preparing data after merging with Echonest features
X_EXCLUDE = [
    'Unnamed: 0', 
    'track_id',
    'duration',           
    'duration_clipped',   
    'y_duration_log',     
    'genres',             
    'genres_all'          
]

Y1 = df_complet['y_duration_log']
X1 = df_complet.drop(columns=[col for col in X_EXCLUDE if col in df_complet.columns], errors='ignore')

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled1 = scaler.fit_transform(X_train1)
X_test_scaled1 = scaler.transform(X_test1)
X_train_scaled1 = pd.DataFrame(X_train_scaled1, columns=X_train1.columns, index=X_train1.index)
X_test_scaled1 = pd.DataFrame(X_test_scaled1, columns=X_test1.columns, index=X_test1.index)


In [20]:
# Apply Random Forest Regressor on the new dataset
model_rf1 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf1.fit(X_train_scaled1, Y_train1)

# Prediction
Y_pred_log1 = model_rf1.predict(X_test_scaled1)

# Evaluation
rmse_log1 = np.sqrt(mean_squared_error(Y_test1, Y_pred_log1))
mae_log1 = mean_absolute_error(Y_test1, Y_pred_log1)

# --- FIX BELOW ---
# Originally used 'mae_log' (from the previous experiment) instead of 'mae_log1'
mae_seconds1 = np.expm1(mae_log1) 
r2_score1 = model_rf1.score(X_test_scaled1, Y_test1)

print("\nPerformance Random Forest Regressor after merging with Echonest features:")
print(f"R² Score: {r2_score1:.4f}")
print(f"RMSE (Log-Seconds): {rmse_log1:.4f}")
print(f"MAE (Log-Seconds): {mae_log1:.4f}")
print(f"MAE (Seconds): {mae_seconds1:.2f} seconds")


Performance Random Forest Regressor after merging with Echonest features:
R² Score: 0.3205
RMSE (Log-Seconds): 0.4343
MAE (Log-Seconds): 0.3169
MAE (Seconds): 0.37 seconds


## Gradient Boosting

In [21]:
model_gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model_gbr.fit(X_train_scaled1, Y_train1)

# Predictions
Y_pred_log2 = model_gbr.predict(X_test_scaled1)
r2_score2 = model_gbr.score(X_test_scaled1, Y_test1)
rmse_log2 = np.sqrt(mean_squared_error(Y_test1, Y_pred_log2))
mae_log2 = mean_absolute_error(Y_test1, Y_pred_log2)

# --- FIX BELOW ---
# Originally used 'mae_log' instead of 'mae_log2'
mae_seconds2 = np.expm1(mae_log2) 

print("\nPerformance of Gradient Boosting Regressor on the merged dataset:")
# Updated all variables below to use the '2' suffix
print(f"R² Score: {r2_score2:.4f}")
print(f"RMSE (Log-Seconds): {rmse_log2:.4f}")
print(f"MAE (Log-Seconds): {mae_log2:.4f}")
print(f"MAE (Seconds): {mae_seconds2:.2f} seconds")


Performance of Gradient Boosting Regressor on the merged dataset:
R² Score: 0.3457
RMSE (Log-Seconds): 0.4262
MAE (Log-Seconds): 0.3118
MAE (Seconds): 0.37 seconds


In [22]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# 1. Define the parameter grid 
param_grid = {
    'n_estimators': [100, 300, 500, 1000],        # Number of trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],      # How fast it learns (lower is usually better but slower)
    'max_depth': [3, 5, 7, 9],                    # How deep each tree is (deeper = risks overfitting)
    'subsample': [0.6, 0.8, 1.0],                 # % of rows used per tree
    'colsample_bytree': [0.6, 0.8, 1.0],          # % of columns used per tree
    'gamma': [0, 0.1, 0.5],                       # Minimum loss reduction required to make a split
    'reg_alpha': [0, 0.1, 1, 10]                  # L1 Regularization (helps with noise)
}

# 2. Initialize the base model
xgb_base = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)

# 3. Setup the Search
random_search = RandomizedSearchCV(
    estimator=xgb_base, 
    param_distributions=param_grid, 
    n_iter=50, 
    scoring='neg_mean_squared_error', # It tries to minimize error
    cv=3,                             # 3-fold cross-validation (validates on different chunks of data)
    verbose=1, 
    random_state=42, 
    n_jobs=-1                         # Use all processor cores
)


random_search.fit(X_train_scaled1, Y_train1)

# 5. Get the best results
print("\nBest Parameters Found:")
print(random_search.best_params_)

print(f"\nBest RMSE score from search: {np.sqrt(-random_search.best_score_):.4f}")

Starting Hyperparameter Tuning...
Fitting 3 folds for each of 50 candidates, totalling 150 fits

Best Parameters Found:
{'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.8}

Best RMSE score from search: 0.4315


In [27]:


# configuration
# n_estimators: Number of trees (1000 is a good starting point with early stopping, but 500 is fine here)
# learning_rate: Low rate (0.05) makes the model more robust but slower to train
# max_depth: Keep it shallow (3-6) to prevent overfitting
print("\n===== XGBoost Regressor =====")

model_xgb = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    gamma = 0.1, 
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

model_xgb.fit(X_train_scaled1, Y_train1)
evaluate_model(model_xgb, X_test_scaled1, Y_test1)



===== XGBoost Regressor =====
R² Score: 0.4018
RMSE (Log-Seconds): 0.4075
MAE (Log-Seconds): 0.2972
MAE (Seconds): 0.35 seconds

